# Laborator I

## Tehnici de compilare

In cursul acestui laborator vom studia si implementa algoritmi care sunt folositi de compilatoare.

Dupa cum stiti, cand folosim un limbaj **static** de nivel inalt (C, C++, Java etc.) voi scrieti **cod sursa** intr-un fisier (de ex: main.c, Vector.cpp, WebBrowser.java) pe care il pasati unui **compilator** care citeste fisierul si incearca sa vada daca are sens. 

Dupa aceea, daca nu apar erori de compilare (de ex: int a = "ABC";), compilatorul produce un **fisier executabil**. Voi rulati acel fisier si in cazul in care output-ul este gresit, modificati codul sursa, urmand sa compilati din nou fisierul si sa rulati noul executabil.

La acest laborator vom studia doua parti importante ale compilatoarelor: **analiza lexicala** si **analiza sintactica**.

### Analiza lexicala

**DEX**: LÉXIC s. n. ***Totalitatea cuvintelor*** dintr-o limbă; vocabular.

Pentru ca un compilator sa stie cum sa **parseze** un fisier text, el trebuie sa aiba o lista a cuvintelor cheie care sunt folosite de limbaj (ex: <a href="https://www.w3schools.in/c-tutorial/keywords/">C Keywords</a>, <a href="https://www.w3schools.in/cplusplus-tutorial/keywords/">C++ Keywords</a>, <a href="https://www.w3schools.com/python/python_ref_keywords.asp">Python Keywords</a>).

Analiza lexicala consta in parcurgerea cuvintelor din codul sursa si <b>tokenizarea</b> acestora. Prin tokenizare intelgem asocierea unui **tip** pentru fiecare cuvand din codul sursa. De exemplu:

Cod sursa:
> int a = 123;

Tokeni:
<ol>
  <li>"int" = KEYWORD</li>
  <li>"a" = IDENTIFIER</li>
  <li>"=" = OPERATOR</li>
  <li>"123" = LITERAL</li>
  <li>";" = PUNCTUATOR</li>
</ol>
  
Scopul analizei lexicale este de a sparge codul sursa in token si de a-i pasa analizatorului semantic.
    
### Analiza sintactica

**DEX**: SINTÁXĂ, sintaxe, s. f. Parte a gramaticii care studiază funcțiile cuvintelor și ale propozițiilor în vorbire și care ***stabilește regulile de îmbinare*** a cuvintelor în propoziții și a propozițiilor în fraze.

Analiza sintatica se ocupa cu parsarea tokenilor si verificarea sintactica a codului sursa (daca are sens inlantuirea tokenilor). Daca primim de la analizatorul lexical (KEYWORD IDENTIFIER OPERATOR LITERAL PUNCTUATOR => ar putea fi o instructiune). In schimb daca avem: LITERAL IDENTIFIER LITERAL PUNCTUATOR (de ex: 1233 radical 11;) nu are sens din punct de vedere sintactic, deci la acea linie exista o eroare.

### Alte componente

Exista si alte parti ale compilatoarelor pe care nu le vom discuta (de ex: analiza semantica -> observarea daca nu se poate ajunge niciodata undeva in cod (de ex: if (true) { ... } else { /\* nu se va executa \*/ ... }) si partea de optimizare (void f() { int i; for (i = 0; i < 100; i++) i = i + 1 - 1; **return 42;** }; for-ul nu are sens)

## Notare

La acest laborator veti avea de facut doua proiecte:

<ul>
    <li>Proiect analiza lexicala sau algoritmi</li>
    <li>Proiect analiza sintactica sau algoritmi</li>
</ul>

Notarea:
<ul>
    <li>Daca nu prezentati nici un proiect, aveti nota 4</li>
    <li>Daca prezentati un proiect si acesta este complet aveti nota 7</li>
    <li>Daca prezentati doua proiecte si acestea sunt complete aveti nota 10</li>
</ul>

### Fiecare proiect are 3 puncte!

### SURSELE VOR FI TESTATE ANTIPLAGIAT!

Fiecare student va trebui sa incarce pe Github un folder (cu numele proiectului ales pe Google Sheets) avand urmatoarele componente:
<ul>
    <li>un fisier de tip <b>readme</b>, care va contine atat instructiuni de rulare ale programului, structura inputului, un exemplu de input si de output, cat si partile principale ale programului</li>
    <li>un fisier de tip <b>makefile</b>, pentru a putea rula usor fisierul</li>
    <li>un fisier de <b>intrare</b> si un fisier de <b>iesire</b></li>
    <li>fisierele care contin <b>codul sursa</b></li>
    <li>!! se va incarca doar codul sursa, nu si executabilul !!</li>
</ul>

### Fiecare proiect de algoritmica trebuie sa contina in output pasii intermediari <i>importanti</i>!

Pentru fiecare proiect aveti mai multe variante de exercitii:
<ol>
    <li>trebuie sa alegeti un proiect din prima parte I (lexic) si un proiect din a doua parte II (sintactic)</li>
    <li>proiecte care tin de analiza lexicala sau sintactica: (<b>I.1, I.2, I.3</b> sau <b>II.2, II.3, II.4</b>)</li>
    <li>proiecte de algoritmica: (<b>I.4, I.5</b> sau <b>II.1, II.5, II.6</b>, II.7, II.8, II,9)</li>
</ol>

**Analiza lexicala sau algoritmi (maxim 5 persoane per proiect)**:
<ol>
    <li>Construirea unui lexer <b>manual</b> pentru un <i>limbaj de programare</i> folosind automate finite deterministe (DFA)</li>
    <li>Construirea unui lexer <b>automat</b> pentru un <i>limbaj de programare</i> folosind lex sau o reimplementare a acestuia</li>
    <li>Construirea unui lexer (<b>reimplementare lex</b>) folosind expresii regulate (<b>Regex</b>)</li>
    <li>Translator stiva nedeterminist</li>
    <li>Convertire expresie regulata in automat finit determinist (REGEX => DFA)</li>
</ol>

#### Analizator lexical (manual sau automat)

Pentru prima tema trebuie sa scrieti un analizor lexical pentru un limbaj de programare la alegere. Scrieti analizorul sub forma unei functii care returneaza: 

<ul>
    <li>tipul token-ului curent</li>
    <li>lungimea sirului corespunzator din fisierul de intrare</li>
    <li>linia din fisierul de intrare pe care se afla token-ul curent</li>
    <li>pointer catre primul caracter al token-ulului curent</li>
    <li>un mesaj de eroare atunci cand este intalnita o eroare lexicala.</li>
</ul>    
    
Functia este apelata din programul principal, in care este citit un fisier de intrare care va fi scanat cu ajutorul acestei functii, astfel incat sa se afiseze toti token-ii care apar in fisierul de intrare. Atunci cand este apelata, functia de scanare:

<ul>
    <li>incepand de la pointerul curent (care initial indica catre primul caracter al fisierului de intrare) sare peste un nr de caractere egal cu lungimea token-ului anterior (initial aceasta lungime este 0);</li>
    <li>sare peste spatii, tab-uri, linii noi, pana intalneste primul caracter diferit de acestea; seteaza pointerul curent astfel ca sa indice catre acest caracter;</li>
    <li>identifica token-ul curent, ce corespunde sirului ce incepe cu caracterul depistat la pasul anterior; determina tipul acestuia si lungimea sirului corespunzator;</li>
    <li>in cazul in care este intalnita o eroare lexicala, semneleaza aceasta printr-un mesaj, scaneaza fisierul de intrare in continuare, pana gaseste primul caracter de tip spatiu, linie noua, tab, seteaza pointerul curent catre acest caracter, seteza lungimea token-ului curent cu 0 (in felul acesta programul va afisa in continuare token-ii urmatori, fara sa se opreasca la prima eroare intalnita).</li>
    <li>se opreste cu scanarea cand a intalnit sfarsitul fisierului de intrare.</li>
</ul>

Pentru cazul **automat** trebuie doar sa returnati pentru fiecare token structura de mai sus.

#### Translator stiva nedeterminist
Program care simuleaza functionarea unui translator stiva nedeterminist cu lambda-tranzitii. Programul citeste dintr-un fisier elementele unui translator stiva nedeterminist cu lambda-tranzitii oarecare (starile, starea initiala, starile finale, alfabetul de intrare, alfabetul de iesire, alfabetul stivei, simbolul initial al stivei, tranzitiile). Programul permite citirea unui nr. oarecare de siruri peste alfabetul de intrare al translatorului. Pentru fiecare astfel de sir se afiseaza toate iesirile (siruri peste alfabetul de iesire) corespunzatoare (Atentie! pot exista 0, 1 sau mai multe iesiri pt acelasi sir de intrare).

#### REGEX => DFA (arbori)
Primiti intr-un fisier text o expresie regulata care contine litere (a, b, c etc.), operatorii expresiilor regulate (| -> sau, \* -> Kleene star) si paranteze si returneaza un DFA care recunoaste aceleasi cuvinte ca si expresia regulata primita. Exemplu de input: **(a|b)\*abb**, output: automatul finit care recunoaste cuvintele care se termina cu **abb**. La aceasta problema trebuie sa afisati DFA-ul printr-un graf (text) cat si grafic (folosind eventual pachete existente ca **networkx** sau altele). Daca nu afisati si grafic, vi se va scadea 1 punct.

**Analiza sintactica sau algoritmi (maxim 5 persoane per proiect)**:
<ol>
    <li>Algoritmul CYK</li>
    <li>Recursive descent parser</li>
    <li>DSL, folosind lex si yacc</li>
    <li>Autoformatare pentru un limbaj de programare</li>
    <li>Parser LL(1)</li>
    <li>Parser LR(0)</li>
    <li>Parser LL(k) <b>+1 pct</b></li>
    <li>Parser LR(1) <b>+1 pct</b></li>
    <li>Parser LALR(1) <b>+1 pct</b></li>
</ol>


#### Algoritmul CYK
Primiti intr-un fisier o gramatica independenta de context in forma normala Chomsky si o lista de cuvinte. Trebuie sa folositi algoritmul CYK pentru a spune daca fiecare cuvant poate fi generat de gramatica si in cazul in care se poate, trebuie sa afisati o derivare a acestuia (de ex: S => AA => ABA => ACCA => acca). La aceasta problema trebuie sa afisati atat derivarea cat si grafic (folosind eventual pachete existente ca **networkx** sau altele). Daca afisati si grafic, primiti 1 punct in plus. Daca afisati corect doar daca poate fi generat sau nu, fara derivare, vi se vor scadea 2 puncte.

#### Recursive descent parser
Primiti intr-un fisier text o gramatica si intr-un alt fisier o lista de cuvinte. Trebuie sa generati un cod care atunci cand este rulat, va parsa cel de-al doilea fisier si va indica daca este corect din punct de vedere sintactic. De exemplu scrieti un fisier in Python, care atunci cand primeste o gramatica, scrie intr-un fisier rdp.c cod de C cu tot cu main, il compileaza, si atunci cand este rulat, parseaza al doilea fisier.

#### DSL, folosind lex si yacc
La aceasta problema trebuie sa implementati un Domain Specific Language. Trebuie sa va definiti un limbaj de programare pe care sa il folositi pentru rezolvarea altor probleme. Daca alegeti aceasta problema trebuie sa discutam limbajul si sa va modific putin sintaxa. De exemplu sa presupunem ca vreti sa implementati limbajul C si pe langa acesta vreti o instructiune de tipul:

> fore (x = a[0,n]) printf("%d", x);

Evident, nu exista fore in C, dar daca il intalniti, voi il puteti translata in:

int i, x;

for (i = 0; i < n; i++) { x = a[i]; printf("%d", x); }

#### Autoformatare pentru un limbaj de programare
La aceasta problema intrarea este un fisier care contine cod (\*.c sau \*.java etc.) si voi trebuie sa il rescrieti frumos. De exemplu daca aveti un cod care initial arata asa:

> #include<stdio.h> int main(int argc &emsp;,char** &nbsp;&nbsp;&nbsp;argv){int a=23; if(a>2) {a++;} &nbsp;&nbsp;return a;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;}

Dupa ce apelati programul vostru cu acel input, trebuie sa afisati:

#include<stdio.h> 

int main(int argc, char** argv) { 

&emsp;int a = 23;

&emsp;if(a > 2) {

&emsp;&emsp; a++;

&emsp;}

&emsp;return a;

}

#### Parser (LL(1), LR(0), LL(k), LR(1), LALR(1))
La aceste probleme primiti o gramatica independenta de context (nu neaparat in forma normala Chomsky) si un cuvant si trebuie sa spuneti daca acel cuvant este recunoscut de gramatica folosind acel parser. De asemenea, trebuie sa afisati si pasii intermediari ai algoritmului pentru a putea verifica corectitudinea partiala a programului. La aceste probleme trebuie sa aratati si cum functioneaza parsarea unui cuvant (algoritmul care foloseste stiva).

### Proiecte alternative:

#### Administrator website + 1 singur proiect

Un singur student per grupa poate alege sa se ocupe de un **website** care va descarca proiectul de pe Github si va face o interfata pentru a putea vedea si verifica mai usor proiectele. Daca exista mai multi studenti care vor sa administreze site-ul, acestia vor face o versiune preliminara a acestuia, si va fi ales cel care are interfata mai clara. Daca acel student va alege sa se ocupe de website, acesta va trebui obligatoriu sa mai aleaga un singur proiect din cele de la I si II (nu puteti primi nota fara sa faceti un proiect legat de TC).

Administratorul site-ului va lua legatura cu studentii grupei si le va specifica modul in care trebuie sa salveze fisiere (daca au o singura intrare, daca au mai multe intrari, daca au poze etc.) in directorul lor de pe Github. Acesta va face si interfata grafica in care va specifica: un *readme* care spune cum trebuie dat input-ul - care se va lua de pe Github, un exemplu de input scris deja, eventual un form, care la submit afiseaza pe aceiasi pagina si output-ul (fie doar text, fie si cu imagini, depinzand de problema aleasa), va pune timeout pe executabil (in cazul buclelor infinite etc.).

#### Cercetare

Implementare parser pentru o expresie regulata bidimensionala si afisarea acesteia pe ecran. Daca sunt persoane interesate vom discuta mai multe la laborator despre aceasta problema.

## Instalare sly

Pentru a instala python-sly (care este o reimplementare a programelor lex si yacc) trebuie sa aveti instalat python3 si apoi trebuie sa instalati pachetul sly: 

```$ sudo apt-get update && sudo apt-get install python3.7 python3-pip && sudo pip install sly```

## Verificare instalare

Pentru a verifica instalarea rulati urmatorul cod:

In [42]:
import sys
from sly import Lexer, Parser

print("Python version:", sys.version)

Python version: 3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


## Constructia unui lexer simplu

Pentru a construi un lexer simplu trebuie sa definim o clasa si sa derivam din clasa Lexer. Dupa aceasta trebuie sa definim tipurile de tokeni folositi si sa definim o expresie regulata pentru fiecare dintre acestia. Apoi trebuie sa rulam lexer-ul pentru a primi fiecare token dintr-un input.

Ca o prima problema, sa spunem ca vrem sa citim fie numere fie cuvinte si atunci cand am ajuns la un spatiu sa spunem daca ceea ce am citit este un cuvant sau un numar.

In [14]:
class WordOrNumberLexer(Lexer):
    tokens = { WORD, NUMBER }   # avem doar doua tipuri de tokeni
    ignore = ' \t'              # nu ne intereseaza spatiile sau tab-ul

    # Expresii regulate pentru fiecare tip de token
    WORD = r'[a-zA-Z_][a-zA-Z0-9_]*'
    NUMBER = r'\d+'

    # Puteti sa ignorati si pattern-uri, nu doar litere simple cum avem mai sus
    ignore_newline = r'\n+'

    # Puteti sa numarati cate newline-uri aveti in fisierul text
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    # Daca apare altceva inafara de expresiile regulate definite mai sus
    # afisam o eroare intrucat nu putem parsa mai departe.
    # Programul de mai jos doar trece peste aceasta eroare.
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

Avand clasa definita, urmeaza sa implementam functia main care citeste de la tastatura input-uri si afiseaza pentru fiecare dintre acestea in ce categorie e.

In [21]:
lexer = WordOrNumberLexer()
while True:
    try:
        text = input('calc > ')
    except KeyboardInterrupt:
        break
    if text:
        for token in lexer.tokenize(text):
            print(token)

calc > 1231
Token(type='NUMBER', value='1231', lineno=1, index=0)
calc > 123 kjkj 1211 1  jk jlj   kj lj lkj kllll
Token(type='NUMBER', value='123', lineno=1, index=0)
Token(type='WORD', value='kjkj', lineno=1, index=4)
Token(type='NUMBER', value='1211', lineno=1, index=9)
Token(type='NUMBER', value='1', lineno=1, index=14)
Token(type='WORD', value='jk', lineno=1, index=17)
Token(type='WORD', value='jlj', lineno=1, index=20)
Token(type='WORD', value='kj', lineno=1, index=26)
Token(type='WORD', value='lj', lineno=1, index=29)
Token(type='WORD', value='lkj', lineno=1, index=32)
Token(type='WORD', value='kllll', lineno=1, index=36)


Sa modificam clasa pentru a numara cate cuvinte si cate numere avem.
De asemenea nu mai citim de la tastatura ci vom presupune ca am citit din fisier.

In [41]:
class WordOrNumberLexer(Lexer):
    tokens = { WORD, NUMBER }   # avem doar doua tipuri de tokeni
    ignore = ' \t'              # nu ne intereseaza spatiile sau tab-ul

    def __init__(self):
        self.words_count = 0
        self.numbers_count = 0
        
        self.words = []
        self.numbers = []
    
    # Expresii regulate pentru fiecare tip de token
    WORD = r'[a-zA-Z_][a-zA-Z0-9_]*'
    NUMBER = r'\d+'
    
    def WORD(self, w):
        self.words_count += 1
        self.words.append(w.value)
        
        return w
        
    def NUMBER(self, n):
        self.numbers_count += 1
        self.numbers.append(n.value)
        
        return n
    
    # Puteti sa ignorati si pattern-uri, nu doar litere simple cum avem mai sus
    ignore_newline = r'\n+'

    # Puteti sa numarati cate newline-uri aveti in fisierul text
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    # Daca apare altceva inafara de expresiile regulate definite mai sus
    # afisam o eroare intrucat nu putem parsa mai departe.
    # Programul de mai jos doar trece pe aceasta eroare.
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [40]:
input_text = '1337 test other 1122 1 3 here'
lexer = WordOrNumberLexer()

for token in lexer.tokenize(input_text):
    print(token)

print()    
print('Numbers_count = {}, Numbers = {}'.format(lexer.numbers_count, lexer.numbers))
print('Words_count = {}, Words = {}'.format(lexer.words_count, lexer.words))

Token(type='NUMBER', value='1337', lineno=1, index=0)
Token(type='WORD', value='test', lineno=1, index=5)
Token(type='WORD', value='other', lineno=1, index=10)
Token(type='NUMBER', value='1122', lineno=1, index=16)
Token(type='NUMBER', value='1', lineno=1, index=21)
Token(type='NUMBER', value='3', lineno=1, index=23)
Token(type='WORD', value='here', lineno=1, index=25)

Numbers_count = 4, Numbers = ['1337', '1122', '1', '3']
Words_count = 3, Words = ['test', 'other', 'here']


Dupa cum puteti vedea numerele salvate in lista numbers sunt inca string-uri. Daca vreti sa obtineti suma lor trebuie mai intai sa le convertiti la numere intregi (folosind functia int('123')) si apoi de fiecare data cand cititi un numar sa il adaugati la suma. Alternativ, puteti face asta si in programul principal.

### Exercitiu

Primind un for-uri si if-uri afisati tokenii lor. Pentru a usura implementarea la acest moment, vom considera ca avem doua cuvinte cheie: for si if; instructiunile sunt date ca a=b si conditiile a<b; si aveti paranteze si acolade deschise si inchise. De exemplu daca primiti for(i=0; i<9; i=i+1) { if(i>2) a=a+i; if (i<3) a=a+2; } trebuie sa afisati FOR PDESCHISA INSTRUCTIUNE CONDITIE INSTRUCTIUNE PINCHISA ADESCHISA IF PDESCHISA CONDITIE PINCHISA ...

In [55]:
class ForIfLexer(Lexer):
    tokens = { FOR_TOKEN, IF_TOKEN, INSTRUCTION, CONDITION, OPENP, CLOSEDP, OPENCB, CLOSEDCB }
    ignore = ' \t'
    
    # Expresii regulate pentru fiecare tip de token
    FOR_TOKEN = r'for'
    IF_TOKEN = r'if'
    INSTRUCTION = r'[a-zA-Z_][a-zA-Z0-9_]*\=([a-zA-Z_][a-zA-Z0-9_]*|\d+)(\+([a-zA-Z_][a-zA-Z0-9_]*|\d+))?;?'
    CONDITION = r'[a-zA-Z_][a-zA-Z0-9_]*(<|>|<\=|>\=)([a-zA-Z_][a-zA-Z0-9_]*|\d+);?'
    OPENP = r'\('
    CLOSEDP = r'\)'
    OPENCB = r'{'
    CLOSEDCB = r'}'
    
    # Puteti sa ignorati si pattern-uri, nu doar litere simple cum avem mai sus
    ignore_newline = r'\n+'

    # Puteti sa numarati cate newline-uri aveti in fisierul text
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    # Daca apare altceva inafara de expresiile regulate definite mai sus
    # afisam o eroare intrucat nu putem parsa mai departe.
    # Programul de mai jos doar trece pe aceasta eroare.
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [100]:
input_text = 'for(i=0; i<9; i=i+1) { if(i>2) a=a+i; if (i<3) a=a+2; } '
lexer = ForIfLexer()

tokens_value = []
for token in lexer.tokenize(input_text):
    print(token)
    tokens_value.append(token.type)

print()
print(' '.join(tokens_value))

Token(type='FOR_TOKEN', value='for', lineno=1, index=0)
Token(type='OPENP', value='(', lineno=1, index=3)
Token(type='INSTRUCTION', value='i=0;', lineno=1, index=4)
Token(type='CONDITION', value='i<9;', lineno=1, index=9)
Token(type='INSTRUCTION', value='i=i+1', lineno=1, index=14)
Token(type='CLOSEDP', value=')', lineno=1, index=19)
Token(type='OPENCB', value='{', lineno=1, index=21)
Token(type='IF_TOKEN', value='if', lineno=1, index=23)
Token(type='OPENP', value='(', lineno=1, index=25)
Token(type='CONDITION', value='i>2', lineno=1, index=26)
Token(type='CLOSEDP', value=')', lineno=1, index=29)
Token(type='INSTRUCTION', value='a=a+i;', lineno=1, index=31)
Token(type='IF_TOKEN', value='if', lineno=1, index=38)
Token(type='OPENP', value='(', lineno=1, index=41)
Token(type='CONDITION', value='i<3', lineno=1, index=42)
Token(type='CLOSEDP', value=')', lineno=1, index=45)
Token(type='INSTRUCTION', value='a=a+2;', lineno=1, index=47)
Token(type='CLOSEDCB', value='}', lineno=1, index=54)

F

### Exercitiu

Construiti un lexer care primeste numere in baza b (2 <= b <= 10) si **converteste fiecare numar in baza 10**.

Numerele primite sunt de forma baza\\$numar. Daca baza = 10, 10\\$ poate lipsi si poate ramana doar numarul.

Exemple de numere: 2\\$1101 4\\$1132 006\\$5155 10\\$1918. Ultimul numar poate fi dat si ca 1918.


In [94]:
class ConversionLexer(Lexer):
    tokens = {}
    ignore = ' \t'
    
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [93]:
input_text = '2$1101 4$1132 006$5155 10$1918'
lexer = ConversionLexer()

for token in lexer.tokenize(input_text):
    print(token)

AttributeError: type object 'ConversionLexer' has no attribute '_master_re'

### Exercitiu

Notatia condensata poate fi folosita pentru a utiliza mai putina memorie atunci cand dorim sa salvam numere. Principiul condensarii se bazeaza pe faptul ca putem reprezenta un numar relativ la o baza. De exemplu daca avem urmatoarea secventa de numere si baza = 150:

> S = 149 154 154 154 147 147 145 152 153 150 150 148 162

Putem condensa numerele in functie de baza:

> S = 150-1 150+4 150+4 150+4 150-3 150-3 150-5 150+2 150+3 150 150 150-2 150+12

Atunci cand sunt mai multe aparitii ale aceluiasi numar punem + (respectiv -) de cate ori apare numarul respectiv.
Daca numarul este chiar baza punem 0.

> CS = 150\\$-1b+++4b--3b-5b+2b+3b00b-2b+12b

Unde b reprezinta un spatiu.

Voi trebuie sa scrieti un program care va converteste o expresie in din forma CS in S.

In [ ]:
class CondensedNotationLexer(Lexer):
    tokens = {}
    ignore = ' \t'
    
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [95]:
input_text = '150$-1 +++4 --3 -5 +2 +3 00 -2 +12 '
lexer = CondensedNotationLexer()

for token in lexer.tokenize(input_text):
    print(token)

AttributeError: type object 'ConversionLexer' has no attribute '_master_re'